# Text classification using Transformers.
This lab will still focus on text classification on the Imdb dataset.
In this lab session, we will focus on encoder-based transformer architecture, through the lens of the most famous model: **BERT**.

---

# Introduction

## HuggingFace

We have already experimented with some components provided by the HuggingFace library:
- the `datasets` library,
- the `tokenizer`.

Actually, HuggingFace library provides convenient API to deal with transformer models, like BERT, GPT, etc.  To quote their website: *Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce your compute costs, carbon footprint, and save you the time and resources required to train a model from scratch. Transformers support framework interoperability between PyTorch, TensorFlow, and JAX.*

## Goal of the lab session

We will experiment with the HuggingFace library. You'll have to load a model and to run it on your task.

Important things to keep in in minds are:
- Even if each model is a Transformer, they all have their peculiarities.
- What is the exact input format expected by the model?
- What is its exact output?
- Can you use the available model as is or should you make some modifications for your task?

These questions are actually part of the life of a NLP scientist. We will adress some of these questions in this lab and in the next lessons / labs / HW.

In [ ]:
%%capture
!pip install transformers datasets

In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']
import matplotlib.pyplot as plt

from transformers import DistilBertTokenizer
from tqdm.notebook import tqdm


import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
from torch.utils.data import DataLoader
from tabulate import tabulate
from datasets import load_dataset

from tqdm.notebook import tqdm

# If the machine you run this on has a GPU available with CUDA installed,
# use it. Using a GPU for learning often leads to huge speedups in training.
# See https://developer.nvidia.com/cuda-downloads for installing CUDA
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

## Download the training data

In [ ]:
dataset = load_dataset("scikit-learn/imdb", split="train")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

IMDB Dataset.csv:   0%|          | 0.00/66.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})


## Prepare model inputs

The input format to BERT looks like it is  "over-specified", especially if you focus on just one type task: sequence classification, word tagging, paraphrase detection, ...  The format:
- Add special tokens to the start and end of each sentence.
- Pad & truncate all sentences to a single constant length.
- Explicitly differentiate real tokens from padding tokens with the "attention mask".

It looks like that:

<img src="https://drive.google.com/uc?export=view&id=1cb5xeqLu_5vPOgs3eRnail2Y00Fl2pCo" width="600">

If you don't want to recreate this kind of inputs with your own hands, you can use the pre-trained tokenizer associated to BERT. Moreover the function `encode_plus` will:
- Tokenize the sentence.
- Prepend the `[CLS]` token to the start.
- Append the `[SEP]` token to the end.
- Map tokens to their IDs.
- Pad or truncate the sentence to `max_length`
- Create attention masks for `[PAD]` tokens.


> 💡 *Note:* For computational reasons, we will use the [DistilBERT](https://huggingface.co/docs/transformers/model_doc/distilbert) model, which is a 40% smaller than the original BERT model but still achieve about 95% of the performances of the original model.

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained(
    "distilbert-base-uncased", do_lower_case=True
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Let's see how the tokenizer actually process the sequence:

In [ ]:
# Some useful steps:
message = "hello my name is kevin"
tok = tokenizer.tokenize(message)
print("Tokens in the sequence:", tok)
enc = tokenizer.encode(tok)
table = np.array(
    [
        enc,
        [tokenizer.ids_to_tokens[w] for w in enc],
    ]
).T
print("Encoded inputs:")
print(tabulate(table, headers=["Token IDs", "Tokens"], tablefmt="fancy_grid"))

Tokens in the sequence: ['hello', 'my', 'name', 'is', 'kevin']
Encoded inputs:
╒═════════════╤══════════╕
│   Token IDs │ Tokens   │
╞═════════════╪══════════╡
│         101 │ [CLS]    │
├─────────────┼──────────┤
│        7592 │ hello    │
├─────────────┼──────────┤
│        2026 │ my       │
├─────────────┼──────────┤
│        2171 │ name     │
├─────────────┼──────────┤
│        2003 │ is       │
├─────────────┼──────────┤
│        4901 │ kevin    │
├─────────────┼──────────┤
│         102 │ [SEP]    │
╘═════════════╧══════════╛


🚧 **Question** 🚧

You noticed special tokens like `[CLS]` and `[SEP]` in the sequence. Note how they were added automatically by HuggingFace.

- Why are there such special tokens?

**Answer - Edoardo as of 08 2025**

CLS --> model assumes we are working on a classification task. It is a token used to represent the sentence for the next tasks and it can be considered, after several transformer layers, as a summary representation of the sentences.

SEP --> it is the separator token. In this specific case it is not important because we have only 1 sentence. However, it is important when there are several sentences.

The SEP token is very important, for example, in case we have a Q&A tasks.

## Data pre-processing

Usual data-processing for torch. Same as previous lab.

In [ ]:
def preprocessing_fn(x, tokenizer):
    # 1. Convert text to token IDs
    x["input_ids"] = tokenizer.encode(
        x["review"],                # the review text
        add_special_tokens=True,   # modified by Edoardo to generate CLS and SEP tokens (for use later)
        truncation=True,            # cut off if too long
        max_length=256,             # cap length at 256 tokens
        padding=False,              # don’t pad (done later in batching) - pad = filler
        return_attention_mask=False # only return IDs, no mask - attention mask tells the model which token is real vs padded
    )

    # 2. Map sentiment to label
    x["labels"] = 0 if x["sentiment"] == "negative" else 1

    return x

**Explanation Edoardo 08 2025**

The function above converts raw review text and sentiment lables into a tokenized sequence and numerical labels. Those can be consumed by an Hugging Face Model.

In [ ]:
n_samples = 2000  # the number of training example from the dataset

# We first shuffle the data to randomize the examples taken
dataset = dataset.shuffle()

# Select 2000 samples
splitted_dataset = dataset.select(range(n_samples))

# Tokenize the dataset and creates inputs_ids and labels (from previous function)
splitted_dataset = splitted_dataset.map(
    preprocessing_fn, fn_kwargs={"tokenizer": tokenizer}
)

# Remove useless columns / keeps only the generated input_ids and labels
splitted_dataset = splitted_dataset.select_columns(["input_ids", "labels"])

# Split the train and validation
splitted_dataset = splitted_dataset.train_test_split(test_size=0.2)

train_set = splitted_dataset["train"]
valid_set = splitted_dataset["test"]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

**Explanation Edoardo 08 2025**

Prepare data set for model training. Training will be done with DistilBert on 2000 examples randomly selected and tokenized.


In [ ]:
class DataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        features = self.tokenizer.pad(
            batch, padding="longest", max_length=256, return_tensors="pt"
        )
        return features

**Explanation Edoardo 08 2025**

As in the preprocessing function we did not do the padding, we are now doing it. The above code allows to:

1. It takes the input_ids (which are different lengths right now).
2. Pads them so they all match the longest sequence in that batch (efficient, avoids over-padding).
3. Adds an attention mask automatically (1 for real tokens, 0 for padding).
4. Converts everything into PyTorch tensors (torch.LongTensor) so the model can use them.

In [ ]:
# Initiate the collator

data_collator = DataCollator(tokenizer)

**Explanation Edoardo 08 2025**

The collator will:

- Pad the batch dynamically
- Add attention mask labels (0 & 1)
- Return tensors in the right format.

In [ ]:
batch_size = 4 #process 4 examples at a time

train_dataloader = DataLoader(
    train_set, batch_size=batch_size, collate_fn=data_collator, shuffle=True
) #pad and return tensors for the train set, with shuffling
valid_dataloader = DataLoader(
    valid_set, batch_size=batch_size, collate_fn=data_collator
) #pad and return tensors for validation set
n_valid = len(valid_set)
n_train = len(train_set)

**What is done above**

| Step                   | Status | Code you already have                                               | Purpose                                                           |
| ---------------------- | ------ | ------------------------------------------------------------------- | ----------------------------------------------------------------- |
| Dataset loaded         | ✅      | `dataset` already exists                                            | Get raw reviews + sentiments                                      |
| Shuffle                | ✅      | `dataset = dataset.shuffle()`                                       | Remove order bias                                                 |
| Subsample              | ✅      | `dataset.select(range(n_samples))`                                  | Work on 2,000 examples only                                       |
| Tokenization           | ✅      | `dataset.map(preprocessing_fn, fn_kwargs={"tokenizer": tokenizer})` | Convert reviews → token IDs, sentiment → labels                   |
| Column pruning         | ✅      | `select_columns(["input_ids", "labels"])`                           | Keep only relevant fields                                         |
| Train/validation split | ✅      | `train_test_split(test_size=0.2)`                                   | Split into train/valid sets                                       |
| Data collator          | ✅      | `DataCollator(tokenizer)`                                           | Handles **dynamic padding**, adds attention mask, returns tensors |
| Dataloaders            | ✅      | `DataLoader(train_set, ... collate_fn=data_collator)`               | Feed data in mini-batches                                         |


# Add Edoardo 08 2025 (classification)

**What is missing to run a classification using the above**

| Step                                     | What to do                                                                                                                                                            |
| ---------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Device setup & reproducibility**       | Select GPU if available (`device = torch.device("cuda" if torch.cuda.is_available() else "cpu")`), set random seeds for reproducibility (`random`, `numpy`, `torch`). |
| **Load model**                           | Use `DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)`, then move it to `device`.                                         |
| **Optimizer**                            | Define optimizer, usually `AdamW` with small LR (`2e-5` to `5e-5`) and weight decay (e.g., `0.01`).                                                                   |
| **Scheduler (optional but recommended)** | Learning rate warmup + decay, via `get_linear_schedule_with_warmup`.                                                                                                  |
| **Training loop**                        | Iterate over `train_dataloader`: forward pass, compute loss, backward pass, gradient clipping, optimizer + scheduler step.                                            |
| **Validation loop**                      | After each epoch, run `model.eval()` on `valid_dataloader`: compute val loss, accuracy, F1 score, etc.                                                                |
| **Metrics**                              | Track training loss, validation loss, and at least **accuracy** (better also F1 if dataset is imbalanced).                                                            |
| **Logging / monitoring**                 | Print metrics per epoch, optionally add progress bars (`tqdm`).                                                                                                       |
| **Checkpointing**                        | Save best model & tokenizer: `model.save_pretrained("./distilbert-best")`, `tokenizer.save_pretrained("./distilbert-best")`.                                          |
| **Sanity checks**                        | Verify `[CLS]` (id=101) at start, `[SEP]` (id=102) present, `attention_mask` correct; inspect one batch to confirm.                                                   |


## Devide set up & reproducibility

- Ensures results are reproducible across runs (same seed = same shuffling, same weight initialization, etc.).
- Chooses GPU if available, otherwise falls back to CPU.
- Printing confirms where your model will run.

In [ ]:
import torch, random, numpy as np

# Reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


## Load Model

- Loads DistilBERT pretrained on masked language modeling.
- Adds a classification head (a linear layer on top of the [CLS] embedding).
- Sets num_labels=2 → binary classification.
- Moves everything to the device you set (cuda if available). Both the model and the data needs to be on the same devide to interact.

In [ ]:
from transformers import DistilBertForSequenceClassification

# Load pretrained DistilBERT with a classification head (binary sentiment)
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2   # because you have "negative" (0) and "positive" (1)
)

# Move model to device (GPU/CPU)
model = model.to(device)

print("Model loaded with", model.num_labels, "labels")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded with 2 labels


**Interpretation Output**

The model is pretrained for predicting missing words and not for classification.

Also, the model starts from pretrained weights and the classifier must be trained on our sentiment dataset. After this training (fine tuning), the new head learns how to map [CLS] embedding --> positive / negative.

Finally, python suggest us the next step : TRAIN to be able to use the model.

We will:
1. Fine tune
2. Use it

## Optimizer and Scheduler

**Recap Optimizer**

It is an algo that updates the model weights during training (based on the latest computed loss).

Adam learns different learning rates for different weights. The decay prevents overfitting by keeping weights to grow too large.

**Recap Scheduler**

Learning rate = how big the optimizer steps are.

We use the scheduler as the weights are random at the beginning and we need to gradually change them.

From an **intuitive** perspective:

- Optimizer = *how do we adjust the weights based on the loss?*
- Scheduler = *how big should we make the steps from now to the next training?*



In [ ]:
from transformers import get_linear_schedule_with_warmup

# Training hyperparameters
epochs = 3
learning_rate = 2e-5
weight_decay = 0.01

# Optimizer: AdamW is standard for transformers
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay
)

# Total training steps = batches per epoch × epochs
total_steps = epochs * len(train_dataloader)

# Scheduler: linear warmup + decay
warmup_steps = int(0.1 * total_steps)  # 10% warmup
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

print(f"Optimizer and scheduler ready: {total_steps} steps total, {warmup_steps} warmup steps")


Optimizer and scheduler ready: 1200 steps total, 120 warmup steps


## Training & Validation

**Explanation of the theory**

*Forward pass*

Input: a batch of reviews → tokenized into input_ids and attention_mask.

The model:

- Looks up embeddings for each token.
- Passes them through the DistilBERT encoder (stack of Transformer layers).
- Uses the [CLS] token’s hidden state as a representation of the whole sentence.
- Feeds that into the classification head (a small linear layer).
- Output: logits → raw, unnormalized scores for each class (positive/negative).

Mathematically:

$$
\text{logits} = W \cdot h_{\text{[CLS]}} + b
$$

*Loss computation*

We compare the logits with the true labels (0 and 1). We calculate the loss function cross entropy:

$$
L = - \Big( y \cdot \log(\hat{y}) + (1-y) \cdot \log(1-\hat{y}) \Big)
$$

where:

$$
\hat{y}_i = \frac{e^{\text{logits}_i}}{\sum_j e^{\text{logits}_j}}
$$


*Backward pass*

Pytorch automatically computes the gradients of the loss with respect to every model parameters. The gradient (or slope) will tell us how to nugde each weight to reduce the loss.

- If the gradient is +, we should decrease the weight. And viceversa.

*Gradient clipping*

If gradients produced risk to be too large, we can clip them to limit their size and ensure stable updates. Mathematically:

$$
g = min(g,clip  value)
$$

*Optimizer*

Then it is the role of the optimizer to apply the gradients to update weights, including weight decay in our case.

Mathematically:

$$
w \leftarrow w - \eta \cdot \nabla_w L
$$

With n being the learning rate.

*Scheduler*

Instead of keeping the learning rate constant with start small, increase it and then decrease it again.

*Validation*

- Turn off gradients (model.eval() + torch.no_grad()).
- Run the model on validation data.
- Compute validation loss + metrics (accuracy, F1).

This checks if the model is learning general patterns, not just memorizing training data.

*Checkpoint*

Save weights when validation loss improves. To prevent forgetting a good model


In [ ]:
from torch.nn.utils import clip_grad_norm_
import torch

epochs = 3
best_val_loss = float("inf")

for epoch in range(1, epochs + 1):
    print(f"\nEpoch {epoch}/{epochs}")

    # -------- TRAIN --------
    model.train()
    total_train_loss = 0.0

    for batch in train_dataloader:
        # move batch tensors to device
        batch = {k: v.to(device) for k, v in batch.items()}

        optimizer.zero_grad(set_to_none=True)

        # forward pass (returns loss when 'labels' is provided)
        outputs = model(**batch)
        loss = outputs.loss
        total_train_loss += loss.item()

        # backward pass
        loss.backward()
        # gradient clipping for stability
        clip_grad_norm_(model.parameters(), max_norm=1.0)

        # update weights + lr
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"  Training loss: {avg_train_loss:.4f}")

    # -------- VALIDATE --------
    model.eval()
    total_val_loss = 0.0
    correct, total = 0, 0

    with torch.no_grad():
        for batch in valid_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            total_val_loss += loss.item()

            preds = logits.argmax(dim=-1)
            correct += (preds == batch["labels"]).sum().item()
            total += batch["labels"].size(0)

    avg_val_loss = total_val_loss / len(valid_dataloader)
    val_acc = correct / total if total > 0 else 0.0
    print(f"  Validation loss: {avg_val_loss:.4f} | Accuracy: {val_acc:.4f}")

    # save the best checkpoint
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        model.save_pretrained("./distilbert-best")
        tokenizer.save_pretrained("./distilbert-best")
        print("  ✅ Saved new best model to ./distilbert-best")



Epoch 1/3
  Training loss: 0.5283
  Validation loss: 0.4788 | Accuracy: 0.8475
  ✅ Saved new best model to ./distilbert-best

Epoch 2/3
  Training loss: 0.2913
  Validation loss: 0.5518 | Accuracy: 0.8625

Epoch 3/3
  Training loss: 0.1375
  Validation loss: 0.5690 | Accuracy: 0.8725


**Interpretation**

Classic pattern: the model is getting more confident over epochs. Accuracy creeps up, but validation loss rises (overconfidence on wrong cases). This is mild overfitting / miscalibration after epoch 1.

Saving based on the current validation loss, the current best checkpoint is epoch 1. This is the best choice for generalization.


## Sanity Check #1

In [ ]:
# Grab a single batch from the training dataloader
batch = next(iter(train_dataloader))

# Print shapes
print("Input IDs shape:", batch["input_ids"].shape)
print("Attention mask shape:", batch["attention_mask"].shape)
print("Labels shape:", batch["labels"].shape)

# Print the first example
first_input_ids = batch["input_ids"][0]
first_mask = batch["attention_mask"][0]
first_label = batch["labels"][0]

print("\nFirst input_ids:", first_input_ids.tolist())
print("First attention_mask:", first_mask.tolist())
print("First label:", first_label.item())

# Convert IDs back to tokens to inspect
tokens = tokenizer.convert_ids_to_tokens(first_input_ids)
print("\nDecoded tokens:", tokens)


Input IDs shape: torch.Size([4, 256])
Attention mask shape: torch.Size([4, 256])
Labels shape: torch.Size([4])

First input_ids: [101, 1996, 3185, 2838, 2178, 11813, 5792, 2090, 2472, 2520, 1059, 20853, 1998, 19245, 18281, 2000, 3122, 1010, 1996, 2034, 2044, 2000, 3122, 2499, 2006, 6926, 8472, 1012, 2021, 1996, 5848, 1997, 2119, 2122, 2273, 2001, 4208, 2006, 10910, 1037, 6669, 19275, 3185, 1010, 5319, 1999, 1996, 2204, 2214, 2137, 3168, 2004, 1037, 2307, 2466, 1012, 1996, 4087, 5919, 1997, 1996, 3185, 2024, 3139, 2061, 2004, 1996, 13972, 4152, 9063, 2046, 1996, 2895, 2008, 3138, 2173, 2006, 1996, 3898, 2302, 12040, 3436, 2000, 1996, 2373, 1997, 1996, 3746, 1012, 6028, 2003, 2464, 2004, 1037, 4316, 1997, 6630, 4406, 1999, 6926, 8472, 2073, 23447, 1005, 10456, 2806, 2471, 3881, 1996, 3086, 2013, 1996, 2466, 2000, 1996, 2126, 1996, 2466, 2001, 2409, 1012, 2028, 1997, 2026, 5440, 5829, 2007, 2784, 3579, 1999, 2023, 2143, 2003, 1996, 3689, 21527, 2011, 1996, 4192, 2188, 18066, 1997, 11525, 

- Each batch has 4 samples as expected.
- Sequences padded/truncated to length 256, as expected
- Attention mask correctly aligns with input ids
- Special tokens are present



## Sanity Check 2

Validate that special tokens are present in a random batch

In [ ]:
batch = next(iter(train_dataloader))
iid = batch["input_ids"][0]
assert iid[0].item() == tokenizer.cls_token_id, "Missing [CLS]"
assert (iid == tokenizer.sep_token_id).any().item(), "Missing [SEP]"
print("Special tokens OK ✔️")

Special tokens OK ✔️


## Sanity Check 3

Check truncation and padding rates

In [ ]:
import numpy as np
from torch.utils.data import DataLoader

def trunc_pad_stats(dataloader, max_len=256):
    n, n_trunc, n_pad = 0, 0, 0
    for b in dataloader:
        input_ids = b["input_ids"]
        attn = b["attention_mask"]
        # padded examples have any 0 in mask
        n_pad += (attn.sum(dim=1) < input_ids.size(1)).sum().item()
        # truncated examples exactly hit max_len AND have no padding
        n_trunc += ((attn.sum(dim=1) == max_len)).sum().item()
        n += input_ids.size(0)
    return {
        "total_examples": n,
        "padded_frac": n_pad / n,
        "truncated_frac": n_trunc / n,
        "exact_len_frac": (n_trunc / n)  # same as truncated_frac with this logic
    }

stats = trunc_pad_stats(train_dataloader, max_len=256)
print(stats)


{'total_examples': 1600, 'padded_frac': 0.549375, 'truncated_frac': 0.428125, 'exact_len_frac': 0.428125}


*Conclusion check 3*

The truncated fraction is a bit high. This means that we need to raise the max length from 256, otherwise long form context is not retained.

## Sanity Check 4

Spot check a padded example

In [ ]:
def show_padded_example(dataloader):
    for b in dataloader:
        for i in range(b["input_ids"].size(0)):
            attn = b["attention_mask"][i]
            if (attn[-1].item() == 0):  # ends with padding
                ids = b["input_ids"][i]
                toks = tokenizer.convert_ids_to_tokens(ids)
                print("...tokens tail:", toks[-30:])
                print("...mask tail:", attn[-30:].tolist())
                return
    print("No padded example found in this pass")

show_padded_example(train_dataloader)


...tokens tail: ['[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
...mask tail: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


*Conclusion sanity check 4*

Short reviews are padded up to the batch max length and attention mask correctly ignores padding, hence there is no pollution of the context as the model will not read the pad tokens.

## Inference helper

We can, with it, feed raw text and get sentiment predictions from our fine tuned DistilBert



In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch

# Load best saved model + tokenizer
model_path = "./distilbert-best"
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = model.to(device)
model.eval()

def predict_sentiment(text, model=model, tokenizer=tokenizer, device=device):
    # Tokenize the input
    inputs = tokenizer(
        text,
        add_special_tokens=True,
        truncation=True,
        max_length=256,
        padding="max_length",   # pad single example
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]

    # Get predicted label
    pred_label = int(logits.argmax(dim=-1).cpu().item())
    label_str = "positive" if pred_label == 1 else "negative"

    return {
        "text": text,
        "pred_label": label_str,
        "probabilities": {
            "negative": float(probs[0]),
            "positive": float(probs[1])
        }
    }

ADD EXPLANATION OF ABOVE

## Example of use

In [ ]:
print(predict_sentiment("I absolutely loved this movie, it was fantastic!"))
print(predict_sentiment("This was the worst film I have ever seen."))


{'text': 'I absolutely loved this movie, it was fantastic!', 'pred_label': 'positive', 'probabilities': {'negative': 0.010202051140367985, 'positive': 0.9897979497909546}}
{'text': 'This was the worst film I have ever seen.', 'pred_label': 'negative', 'probabilities': {'negative': 0.9629215002059937, 'positive': 0.037078533321619034}}


**Summary**

We now have:

1. Preprocessing & batching
2. Training and validation
3. Save of the best model
4. Inference on new text (single ones, not batch in the case above)

## Batch Inference Helper

In [ ]:
def predict_batch(texts, model=model, tokenizer=tokenizer, device=device, max_length=256):
    """
    Predict sentiment for a batch of texts (list of strings).
    Returns a list of dicts with labels and probabilities.
    """
    # Tokenize the whole batch at once
    inputs = tokenizer(
        texts,
        add_special_tokens=True,
        truncation=True,
        max_length=max_length,
        padding=True,            # pad to longest in batch
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1).cpu().numpy()

    # Decode predictions
    results = []
    for text, prob in zip(texts, probs):
        pred_label = int(prob.argmax())
        label_str = "positive" if pred_label == 1 else "negative"
        results.append({
            "text": text,
            "pred_label": label_str,
            "probabilities": {
                "negative": float(prob[0]),
                "positive": float(prob[1])
            }
        })
    return results


## Example of use

In [ ]:
reviews = [
    "I absolutely loved this movie, it was fantastic!",
    "This was the worst film I have ever seen.",
    "The acting was decent but the story was too slow.",
    "What a masterpiece – I’d watch it again and again!"
]

batch_results = predict_batch(reviews)
for res in batch_results:
    print(res)

{'text': 'I absolutely loved this movie, it was fantastic!', 'pred_label': 'positive', 'probabilities': {'negative': 0.010202061384916306, 'positive': 0.9897979497909546}}
{'text': 'This was the worst film I have ever seen.', 'pred_label': 'negative', 'probabilities': {'negative': 0.9629215002059937, 'positive': 0.03707852587103844}}
{'text': 'The acting was decent but the story was too slow.', 'pred_label': 'negative', 'probabilities': {'negative': 0.9495866894721985, 'positive': 0.05041326582431793}}
{'text': 'What a masterpiece – I’d watch it again and again!', 'pred_label': 'positive', 'probabilities': {'negative': 0.021995682269334793, 'positive': 0.9780043363571167}}


## END Edoardo 08 2025

# Model from scratch

For this task, we will start from a randomly initialized model.

## Retrieve the architecture configuration

In HuggingFace, model's parameters are specified through a `config` file. It is a json-like object.

We can retrieve the one from the official model with the following code:

In [ ]:
from transformers import DistilBertConfig

model_config = DistilBertConfig.from_pretrained("distilbert-base-uncased")
print(model_config)

🚧 **Question** 🚧

Make sure you understand the parameters of the configuration.
- Which ones are task-agnostic parameters?
- Which ones are not?
- Why are there different parameters for different tasks?

**Answer**

TODO



Several architectures are available for DistilBert on HuggingFace, designed for a variety of NLP tasks. Though these interfaces are all built on top of a trained DistilBert model, each has different top layers and output types designed to accomodate their specific NLP task.  

Here is the current list of classes provided for fine-tuning:
* BertModel
* BertForMaskedLM
* BertForNextSentencePrediction
* BertForSequenceClassification
* BertForTokenClassification
* BertForQuestionAnswering

The documentation for these can be found under [here](https://huggingface.co/docs/transformers/model_doc/distilbert).




🚧 **TODO** 🚧

For our first experiment, we want to build from a standard stack of transformer layers, without any additional task-specific head.

Which architecture is the corresponding one?

Choose the right one and initialize the model below, with the config.

In [ ]:
from transformers import # TODO

bert = # TODO

In [ ]:
print(bert)

Just for curiosity's sake, we can browse all of the model's parameters by name here.

In the below cell, we printed out the names and dimensions of the weights for:

- The embedding layer
- The first of the twelve transformers
- The output layer.



In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(bert.named_parameters())

In [ ]:
print("The BERT model has {:} different named parameters.\n".format(len(params)))

print("==== Embedding Layer ====\n")

for p in params[0:4]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print("\n==== First Transformer Layer ====\n")

for p in params[4:20]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

🚧 **TODO** 🚧

Test your bert.
We can already try the model on the validation set. Before just look at the output of the model on one batch.
- Interpret the output.  
- Do you understand everything ?


In [ ]:
batch = next(iter(train_dataloader))

input_ids = batch["input_ids"]
attention_mask = batch["attention_mask"]

output = # TODO
print(output["last_hidden_state"].shape)

## Building a classifier

Our `bert` model is simply a stack of transformer layers. We would like to use it as a backbone for text classification.

🚧 **TODO** 🚧

Wraps the model into a classifier.

> 💡 *Hint*: Use the last hidden [CLS] vector representation to perform classification.

In [ ]:
class DistilBertClassifier(nn.Module):
    def __init__(self, bert):
        super().__init__()
        self.bert = bert
        self.drop = nn.Dropout(0.3)
        self.linear = nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask, **kwargs):
        # TODO
        return output

In [ ]:
bert = DistilBertModel(model_config)
model = DistilBertClassifier(bert)
model.to(DEVICE)

🚧 **TODO** 🚧

Test your model on the batch.
Make sure it has the right shape.

In [ ]:
out = model(
    input_ids=batch["input_ids"].cuda(), attention_mask=batch["attention_mask"].cuda()
)
print(out.shape)

### Training

🚧 **TODO** 🚧

Train your model.
Make sure you track the following quantities per epoch:
- training loss
- training accuracy
- validation loss
- validation accuracy

In [ ]:
def validation(model, valid_dataloader):
    total_size = 0
    acc_total = 0
    loss_total = 0
    criterion = nn.BCEWithLogitsLoss()
    model.eval()
    with torch.no_grad():
        for batch in tqdm(valid_dataloader):
            batch = {k: v.cuda() for k, v in batch.items()}
            input_ids = batch["input_ids"]
            labels = batch["labels"]
            attention_mask = batch["attention_mask"]
            preds = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(preds.squeeze(), labels.float().squeeze())
            acc = (preds.squeeze() > 0) == labels
            total_size += acc.shape[0]
            acc_total += acc.sum().item()
            loss_total += loss.item()
    model.train()
    return loss_total / len(valid_dataloader), acc_total / total_size


validation(model, valid_dataloader)

In [ ]:
def training(model, n_epochs, train_dataloader, valid_dataloader, lr=5e-5):
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        eps=1e-08,
    )
    list_val_acc = []
    list_train_acc = []
    list_train_loss = []
    list_val_loss = []
    criterion = nn.BCEWithLogitsLoss()
    for e in range(n_epochs):
        # ========== Training ==========

        # Set model to training mode
        model.train()
        model.to(DEVICE)

        # Tracking variables
        train_loss = 0
        epoch_train_acc = 0
        for batch in tqdm(train_dataloader):
            batch = {k: v.to(DEVICE) for k, v in batch.items()}
            input_ids, attention_mask, labels = (
                batch["input_ids"],
                batch["attention_mask"],
                batch["labels"],
            )
            optimizer.zero_grad()
            # Forward pass
            outputs = # TODO
            loss = # TODO

            # Backward pass
            # TODO

            # Optimization
            # TODO

            train_loss += loss.detach().cpu().item()
            acc = (outputs.squeeze() > 0) == labels.squeeze()
            epoch_train_acc += acc.float().mean().item()
        list_train_acc.append(100 * epoch_train_acc / len(train_dataloader))
        list_train_loss.append(train_loss / len(train_dataloader))

        # ========== Validation ==========

        l, a = validation(model, valid_dataloader)
        list_val_loss.append(l)
        list_val_acc.append(a * 100)
        print(
            e,
            "\n\t - Train loss: {:.4f}".format(list_train_loss[-1]),
            "Train acc: {:.4f}".format(list_train_acc[-1]),
            "Val loss: {:.4f}".format(l),
            "Val acc:{:.4f}".format(a * 100),
        )
    return list_train_loss, list_train_acc, list_val_loss, list_val_acc

In [ ]:
bert = DistilBertModel(model_config)
model = DistilBertClassifier(bert)
model.to(DEVICE)

In [ ]:
batch_size = 16

train_dataloader = DataLoader(
    train_set, batch_size=batch_size, collate_fn=data_collator
)
valid_dataloader = DataLoader(
    valid_set, batch_size=batch_size, collate_fn=data_collator
)
n_valid = len(valid_set)
n_train = len(train_set)

In [ ]:
list_train_loss, list_train_acc, list_val_loss, list_val_acc = training(
    model, 3, train_dataloader, valid_dataloader
)

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[1].plot(list_train_acc, label="Train accuracy")
axs[1].plot(list_val_acc, label="Validation accuracy")
axs[0].plot(list_train_loss, label="Train loss")
axs[0].plot(list_val_loss, label="Validation loss")
axs[0].set_title("Loss")
axs[1].set_title("Accuracy")
axs[0].legend()
axs[1].legend()
plt.legend()
plt.show()

🚧 **Question** 🚧

How does it compare with your convolution model from previous lab?


## Pre-trained model

Now we are going to compare with a pre-trained model.

First, we are going to load the model's weights from the HuggingFace hub.

In [ ]:
bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
model = DistilBertClassifier(bert)
model.to(DEVICE)

## Fine-Tuning

With our model loaded and ready,  we need to grab the training hyperparameters from within the stored model.

For the purposes of fine-tuning, the authors recommend choosing from the following values (from Appendix A.3 of the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf)):

- **Batch size:** 16, 32  
- **Learning rate (Adam):** 5e-5, 3e-5, 2e-5  
- **Number of epochs:** 2, 3, 4

We chose:
* Batch size: 16 (set when creating our DataLoaders)
* Learning rate: 5e-5
* Epochs: 3 (we'll see that this is probably too many...)

The epsilon parameter `eps = 1e-8` is "a very small number to prevent any division by zero in the implementation" (from [here](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)).

You can find the creation of the AdamW optimizer in `run_glue.py` [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L109).

🚧 **TODO** 🚧

Build the classifier and train it with the pre-trained checkpoint.

In [ ]:
list_train_loss, list_train_acc, list_val_loss, list_val_acc = # TODO

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[1].plot(list_train_acc, label="Train accuracy")
axs[1].plot(list_val_acc, label="Validation accuracy")
axs[0].plot(list_train_loss, label="Train loss")
axs[0].plot(list_val_loss, label="Validation loss")
axs[0].set_title("Loss")
axs[1].set_title("Accuracy")
axs[0].legend()
axs[1].legend()
plt.legend()
plt.show()

🚧 **Question** 🚧

What do you think of the results?

**Answer**

TODO